# What is lazy evaluation?

call when you need :)

In [1]:
import functools

In [2]:
# This is equall with python cached_property
class lazy_property:
    def __init__(self, function):
        self.function = function
        functools.update_wrapper(self, function)
        
    def __get__(self, obj, type_):
        if obj is None:
            return self
        val = self.function(obj)
        obj.__dict__[self.function.__name__] = val
        return val

In [3]:
def lazy_property2(fn):
    attr = '_lazy__' + fn.__name__

    @property
    def _lazy_property(self):
        if not hasattr(self, attr):
            setattr(self, attr, fn(self))
        return getattr(self, attr)

    return _lazy_property

In [4]:
class Person:
    def __init__(self, name, occupation):
        self.name = name
        self.occupation = occupation
        self.call_count2 = 0

    @lazy_property
    def relatives(self):
        # Get all relatives, let's assume that it costs much time.
        relatives = "Many relatives."
        return relatives

    @lazy_property2
    def parents(self):
        self.call_count2 += 1
        return "Father and mother"

In [5]:
def main():
    """
    >>> Jhon = Person('Jhon', 'Coder')
    >>> Jhon.name
    'Jhon'
    >>> Jhon.occupation
    'Coder'
    # Before we access `relatives`
    >>> sorted(Jhon.__dict__.items())
    [('call_count2', 0), ('name', 'Jhon'), ('occupation', 'Coder')]
    >>> Jhon.relatives
    'Many relatives.'
    # After we've accessed `relatives`
    >>> sorted(Jhon.__dict__.items())
    [('call_count2', 0), ..., ('relatives', 'Many relatives.')]
    >>> Jhon.parents
    'Father and mother'
    >>> sorted(Jhon.__dict__.items())
    [('_lazy__parents', 'Father and mother'), ('call_count2', 1), ..., ('relatives', 'Many relatives.')]
    >>> Jhon.parents
    'Father and mother'
    >>> Jhon.call_count2
    1
    """


if __name__ == "__main__":
    import doctest
    doctest.testmod(optionflags=doctest.ELLIPSIS)

**********************************************************************
File "__main__", line 6, in __main__.main
Failed example:
    Jhon.occupation
Expected:
    'Coder'
    # Before we access `relatives`
Got:
    'Coder'
**********************************************************************
File "__main__", line 11, in __main__.main
Failed example:
    Jhon.relatives
Expected:
    'Many relatives.'
    # After we've accessed `relatives`
Got:
    'Many relatives.'
**********************************************************************
1 items had failures:
   2 of  10 in __main__.main
***Test Failed*** 2 failures.
